In [1]:
!pip install psycopg2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.5/383.5 kB 338.2 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for psycopg2: filename=psycopg2-2.9.7-cp311-cp311-linux_x86_64.whl size=168892 sha256=04c1afbd49a2fa677475426436148b50d9335a54b5eababe1402dfeb8b528ea0
  Stored in directory: /home/jovyan/.cache/pip/wheels/ca/a2/1f/9f85470e2065e7a4089baa6c47cfa57a2f8f8e78e07390c5b4
Successfully built psycopg2


In [2]:
import psycopg2

# データベース接続情報
host = 'postgis'
port = '5432'
user = 'postgres'
password = 'postgres'
database = 'sandbox'

# PostGISへの接続
connection = psycopg2.connect(
    host=host,
    port=port,
    user=user,
    password=password,
    database=database
)

# クエリの実行
cursor = connection.cursor()

# getTableAndColumnsName
schema = "public"
query = f"""
SELECT
  t.table_name,
  c.*
FROM
  information_schema.tables t
    JOIN information_schema.columns c
      ON t.table_name = c.table_name
WHERE
  t.table_schema = '{schema}'
    AND c.table_schema = '{schema}'
ORDER BY
  t.table_name,
  c.ordinal_position;
"""

# クエリの実行と結果の取得
cursor.execute(query)
results = cursor.fetchall()

# 結果の表示
table_info = []
for row in results:
    table_name = row[3]
    if table_name == "geography_columns":
        continue
    if table_name == "geometry_columns":
        continue
    if table_name == "spatial_ref_sys":
        continue
    column_name = row[4]
    nullable = row[7]
    data_type = row[8]
    data_type_sub = row[28]
    table_info_row = f"table_name: {table_name}, column_name: {column_name}, data_type: {data_type}/{data_type_sub}, nullable: {nullable}"
    table_info.append(table_info_row)

# コネクションのクローズ
cursor.close()
connection.close()

print("\n".join(table_info))

table_name: flood_areas_akita_2023_07_18, column_name: gridcode, data_type: bigint/int8, nullable: YES
table_name: flood_areas_akita_2023_07_18, column_name: flood_depth, data_type: text/text, nullable: YES
table_name: flood_areas_akita_2023_07_18, column_name: geometry, data_type: USER-DEFINED/geometry, nullable: YES
table_name: flood_areas_akita_2023_07_19, column_name: geometry, data_type: USER-DEFINED/geometry, nullable: YES
table_name: hospitals_akita_2023_07_18, column_name: id, data_type: bigint/int8, nullable: YES
table_name: hospitals_akita_2023_07_18, column_name: name, data_type: character varying/varchar, nullable: YES
table_name: hospitals_akita_2023_07_18, column_name: name_en, data_type: character varying/varchar, nullable: YES
table_name: hospitals_akita_2023_07_18, column_name: geom, data_type: USER-DEFINED/geometry, nullable: YES
table_name: parks, column_name: id, data_type: bigint/int8, nullable: YES
table_name: parks, column_name: name, data_type: character varying

In [3]:
import os

OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
print(len(OPENAI_API_KEY))


51


In [7]:
!pip install langchain openai

  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/67/78/7588a047e458cb8075a4089d721d7af5e143ff85a2388d4a28c530be0494/openai-0.27.8-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.0 MB/s eta 0:00:00a 0:00:01


In [8]:
from langchain import OpenAI, PromptTemplate

llm = OpenAI(model_name="text-davinci-003", temperature=0)

_DEFAULT_TEMPLATE = """Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"

Only use the following tables:

{table_info}

Examples:
Question: 秋田県の浸水エリア内にある公園を教えて
SQLQuery: SELECT p.id, p.name, p.name_en, p.geom FROM parks_akita_2023_07_18 AS p WHERE EXISTS (SELECT 1 FROM flood_areas_akita_2023_07_18 AS f WHERE ST_Intersects(p.geom, f.geometry));
Note: do not forget `SELECT 1` in subquery

Question: {input}"""

prompt_template = PromptTemplate(
    input_variables=["input", "table_info", "dialect"], template=_DEFAULT_TEMPLATE
)

prompt = prompt_template.format(dialect="PostGIS SQL", table_info="\n".join(table_info), input="秋田県の浸水エリア内にある学校を教えて", )

print(prompt)

sql_query = llm(prompt).replace('SQLQuery: ', '').split("\n")[1]
print(sql_query)


Given an input question, first create a syntactically correct PostGIS SQL query to run, then look at the results of the query and return the answer.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"

Only use the following tables:

table_name: flood_areas_akita_2023_07_18, column_name: gridcode, data_type: bigint/int8, nullable: YES
table_name: flood_areas_akita_2023_07_18, column_name: flood_depth, data_type: text/text, nullable: YES
table_name: flood_areas_akita_2023_07_18, column_name: geometry, data_type: USER-DEFINED/geometry, nullable: YES
table_name: flood_areas_akita_2023_07_19, column_name: geometry, data_type: USER-DEFINED/geometry, nullable: YES
table_name: hospitals_akita_2023_07_18, column_name: id, data_type: bigint/int8, nullable: YES
table_name: hospitals_akita_2023_07_18, column_name: name, data_type: character varying/varchar, nullable: YES
table_name: hospitals_akita_2023_07_18, column_name: name_en, data_type: character varying/varcha

In [9]:
# PostGISへの接続
connection = psycopg2.connect(
    host=host,
    port=port,
    user=user,
    password=password,
    database=database
)

# クエリの実行
cursor = connection.cursor()

# クエリの実行と結果の取得
cursor.execute(sql_query)
results = cursor.fetchall()

# 結果の表示
print("結果:", results)

# コネクションのクローズ
cursor.close()
connection.close()

結果: [(387752594, '秋田市立城東中学校', '', '0103000020E6100000010000000F00000096F37B51B184614061600C9BB7DB4340EC8909C5B184614084FC2257A0DB43407E97080AB2846140BCA24A8391DB4340C06F8D19B2846140C35785178EDB434025D9D946B2846140636EADE584DB4340905E1E7CB284614010B8095284DB43400C42D4D8B98461405C4F19A485DB43406B47718EBA84614039036DBC85DB4340A670F37CBC8461404A9C700A86DB434049D57613BC846140F2D82A1C9CDB4340141ED901BC846140DED7929F9EDB434058D647F1BB84614001D64FA4A0DB4340E8D8E66BBB84614053020759BBDB434069633612B2846140D1292389B9DB434096F37B51B184614061600C9BB7DB4340'), (387752600, '秋田市立東小学校', '', '0103000020E61000000100000008000000B3F96EA97984614050262488A9DB434038B5D8927A8461403E4162BB7BDB4340FF1AB5677B84614086C54DB27ADB43408A3E1F658484614026E5EE737CDB43405A2DB0C784846140B4E1557B7DDB434090B9D7EE83846140FC3B365DAADB43409C22765C83846140C6C61748ABDB4340B3F96EA97984614050262488A9DB4340')]
